In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
import time
import numpy as np

In [3]:
icd_url_global = pd.read_csv('datasets/icd_url_global.csv')

In [4]:
# INICIAR DRIVER 
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
driver = webdriver.Firefox(options=options)

In [7]:
# EXTRAER TEXTO
resultados_texto = []
size_1 = 0
size_2 = 100
for url in tqdm(icd_url_global.iloc[size_1:size_2,:].url):
    try:
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        
        # EXTRAER TEXTO
        for script in soup(['script', 'style']):
            script.extract()
        text = soup.get_text()
        
        # ELIMINAR ESPACIOS 
        lines = (line.strip() for line in text.splitlines())
        # SEPARAR MULTI-TITULOS EN UNA LINEA CADA UNO
        chunks = (phrase.strip() for line in lines for phrase in line.split(' '))
        # ELIMINAR LINEAS VACIAS
        text = '\n'.join(chunk for chunk in chunks if chunk)
        resultados_texto.append(text)
    except: 
        resultados_texto.append('Error')
    time.sleep((15-5)*np.random.random()+5)
part_df = icd_url_global.iloc[size_1:size_2,:]
part_df['text'] = resultados_texto

100%|█████████████████████████████████████████████████████████| 100/100 [18:53<00:00, 11.33s/it]
C:\Users\Edson Cepeda\AppData\Local\Temp\ipykernel_18904\1291973857.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part_df['text'] = resultados_texto


In [9]:
part_df.to_csv(f'datasets/icd_part_text_{size_1}_{size_2}.csv', index=False)